In [181]:
import h5py
import numpy as np 
import pandas as pd
import os
import sys
import torch
import tangermeme
from tangermeme.predict import predict
from scipy import stats
from tangermeme.utils import characters


In [189]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with h5py.File('DeepSTARR_data/deepstarr_data.h5', 'r') as dataset:
    print(dataset.keys())
    X_test = np.array(dataset['x_test']).astype(np.float32).transpose(0, 2, 1)
    y_test = np.array(dataset['y_test']).astype(np.float32).transpose(1,0)
    X_train = np.array(dataset['x_train']).astype(np.float32).transpose(0, 2, 1)
    y_train = np.array(dataset['y_train']).astype(np.float32).transpose(1, 0)

test_library_x = torch.tensor(X_test, dtype=torch.float32).to(device)
test_library_y = torch.tensor(y_test, dtype=torch.float32)
train_library_x = torch.tensor(X_train, dtype=torch.float32).to(device)
train_library_y = torch.tensor(y_train, dtype=torch.float32)

print(test_library_x.shape)
print(test_library_y.shape)
print(train_library_x.shape)
print(train_library_y.shape)
print(test_library_x[0:1].shape)



<KeysViewHDF5 ['x_test', 'x_train', 'x_valid', 'y_test', 'y_train', 'y_valid']>
torch.Size([41186, 4, 249])
torch.Size([41186, 2])
torch.Size([402296, 4, 249])
torch.Size([402296, 2])
torch.Size([1, 4, 249])


In [186]:
## load model 
model_dir  = "/grid/wsbs/home_norepl/pmantill/Trained_Model_Zoo/EvoAug_Distilled_Student_Model"
model_path = os.path.join(model_dir, "EvoAug_student_model.pt")

# Make sure distill_EvoAug2.py is in model_dir
sys.path.insert(0, model_dir)
import distill_EvoAug2 as distill_mod  # avoid shadowing 'model'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pytorch_model = torch.load(model_path, map_location=device).to(device).eval()

In [190]:
y = predict(pytorch_model, test_library_x, batch_size=256, func=lambda x: x['mean'])


pearson = [stats.pearsonr(test_library_y[:, i], y[:, i])[0] for i in range(test_library_y.shape[1])]
spearman = [stats.spearmanr(test_library_y[:, i], y[:, i])[0] for i in range(test_library_y.shape[1])]

print(f'Pearson: {pearson}')
print(f'Spearman: {spearman}')
print(f'Mean Pearson: {np.mean(pearson):.4f}')
print(f'Mean Spearman: {np.mean(spearman):.4f}')

Pearson: [0.7012667016098645, 0.7795135789531057]
Spearman: [0.6703389983152167, 0.5966854520896379]
Mean Pearson: 0.7404
Mean Spearman: 0.6335


In [191]:
# get bins

full_library_df = pd.DataFrame({"Sequence": None, "Real_Score_dev": test_library_y[:, 0], "Real_Score_Hk": test_library_y[:, 1], "EvoAug_Score_dev": y[:, 0], "EvoAug_Score_Hk": y[:, 1]}, index=range(test_library_x.shape[0]))

for i in range(test_library_x.shape[0]):
    t = characters(test_library_x[i])
    full_library_df.loc[i, "Sequence"] = t

full_library_df.loc[0, "Sequence"]




'AACATACCCTGCTCTAGCGTATTGCTTTTTAATTGGCAGAAAAATATGTGCTCCGGTCGGTCGGTATTAATATATGTATAAATCCAACCAATATGACACACATTTCATTCAATTTATGACGCACAGCTGACACACTTTTCCCACACTATCTCACCCACATGCATATACACACATACACAGGTTTTTCTGTGCGAAAAATGAATTACGATTCGTCGCGTTTTTTTTTTCTGGTCGCTTGTTTTCGTTTGG'

In [192]:
from tangermeme.utils import one_hot_encode

first_seq = one_hot_encode(full_library_df.loc[0, "Sequence"]).unsqueeze(0).to(device).float()
print(first_seq.shape)


pytorch_model(first_seq)

torch.Size([1, 4, 249])


{'mean': tensor([[2.2068, 0.4771]], device='cuda:0', grad_fn=<AddmmBackward0>),
 'uncertainty': tensor([[0.4180, 0.2331]], device='cuda:0', grad_fn=<SoftplusBackward0>)}

In [193]:
def add_bin(full_library_df):
    # Conditions
    conditions_dev = [
        full_library_df['EvoAug_Score_dev'] <= -1,
        (full_library_df['EvoAug_Score_dev'] >= 0) & (full_library_df['EvoAug_Score_dev'] <= 0.5),
        full_library_df['EvoAug_Score_dev'] >= 2
    ]
    choices = ["Low", "Mid", "High"]

    full_library_df['EvoAug_Score_dev_bin'] = np.select(conditions_dev, choices, default=np.nan)

    conditions_hk = [
        full_library_df['EvoAug_Score_Hk'] <= -1,
        (full_library_df['EvoAug_Score_Hk'] >= 0) & (full_library_df['EvoAug_Score_Hk'] <= 0.5),
        full_library_df['EvoAug_Score_Hk'] >= 2
    ]
    full_library_df['EvoAug_Score_Hk_bin'] = np.select(conditions_hk, choices, default=np.nan)

    return full_library_df

# Apply
full_library_df = add_bin(full_library_df)
full_library_df



,Sequence,Real_Score_dev,Real_Score_Hk,EvoAug_Score_dev,EvoAug_Score_Hk,EvoAug_Score_dev_bin,EvoAug_Score_Hk_bin
0,AACATACCCTGCTCTAGCGTATTGCTTTTTAATTGGCAGAAAAATA...,3.418307,1.983081,2.206788,0.477072,High,Mid
1,ATGCCTGTTGGCCTGCGTAAGGAGCTACCATCAACATCAGCATCGC...,2.211545,-0.379982,0.968712,-0.602406,nan,nan
2,TGTCGCTCCCATTTCGTCAAATGTTGCGTGCTAATTCGCTTGCCTT...,3.483831,1.434863,3.043973,0.577701,High,nan
3,TTGTTGTTGTTGTTGCTCTTGTTGCTGGTGATTTCCACAGGCTTAG...,1.739022,0.147684,1.160347,0.171119,nan,Mid
4,TGTATTGTTGGGTTCCCAATAATTGATTATAAACGAGTGGAAATTG...,4.829350,0.794047,1.473066,-0.021691,nan,nan
...,...,...,...,...,...,...,...
41181,CGGGATTGTCTATTTAAGTCACTCAGCTCCCTTGCTATACCCAAGA...,0.104630,-0.644837,-0.416804,-0.622326,nan,nan
41182,GCACTAGCTGAGTAACAGGTATTTGATCGTTGGGGAACTCTCGTTT...,-1.318970,0.663313,-0.247046,-0.898330,nan,nan
41183,TGAAAGTGTGTGCGTTCTGTTCTCTGTACTTTTCGGTGTAAAAGTA...,0.681030,-2.151505,-0.065238,-0.609813,nan,nan
41184,GCGCCGTGTTAAACACAAGTTTTTTGGCGGAATGCCTATTTAATCT...,1.144431,-1.877330,0.340268,-0.162610,Mid,nan


In [194]:
valid_bins = ["Low", "Mid", "High"]

binned_library_hk = full_library_df[full_library_df['EvoAug_Score_Hk_bin'].isin(valid_bins)]
binned_library_dev = full_library_df[full_library_df['EvoAug_Score_dev_bin'].isin(valid_bins)]

Hk_binned_library = binned_library_hk[["Sequence", "Real_Score_Hk", "EvoAug_Score_Hk", "EvoAug_Score_Hk_bin"]]
Dev_binned_library = binned_library_dev[["Sequence", "Real_Score_dev", "EvoAug_Score_dev", "EvoAug_Score_dev_bin"]]

print(len(binned_library_hk))
print(len(binned_library_dev))


13064
12536


In [195]:
Hk_binned_library.groupby("EvoAug_Score_Hk_bin").count()


,Sequence,Real_Score_Hk,EvoAug_Score_Hk
EvoAug_Score_Hk_bin,,,
High,2893,2893,2893
Low,5355,5355,5355
Mid,4816,4816,4816


In [196]:
Dev_binned_library.groupby("EvoAug_Score_dev_bin").count()

,Sequence,Real_Score_dev,EvoAug_Score_dev
EvoAug_Score_dev_bin,,,
High,3078,3078,3078
Low,2385,2385,2385
Mid,7073,7073,7073


In [226]:
# randomly select 3000 sequences from each bin

hk_low = Hk_binned_library[Hk_binned_library['EvoAug_Score_Hk_bin'] == "Low"].sample(n=2000)
hk_mid = Hk_binned_library[Hk_binned_library['EvoAug_Score_Hk_bin'] == "Mid"].sample(n=2000)
hk_high = Hk_binned_library[Hk_binned_library['EvoAug_Score_Hk_bin'] == "High"].sample(n=2000)

dev_low = Dev_binned_library[Dev_binned_library['EvoAug_Score_dev_bin'] == "Low"].sample(n=2000)
dev_mid = Dev_binned_library[Dev_binned_library['EvoAug_Score_dev_bin'] == "Mid"].sample(n=2000)
dev_high = Dev_binned_library[Dev_binned_library['EvoAug_Score_dev_bin'] == "High"].sample(n=2000)

print(len(hk_low)+len(hk_mid)+len(hk_high))
print(len(dev_low)+len(dev_mid)+len(dev_high))

test_hk_final_df = pd.concat([hk_low, hk_mid, hk_high])
test_dev_final_df = pd.concat([dev_low, dev_mid, dev_high])

test_hk_final_df.groupby("EvoAug_Score_Hk_bin").count()



6000
6000


,Sequence,Real_Score_Hk,EvoAug_Score_Hk
EvoAug_Score_Hk_bin,,,
High,2000,2000,2000
Low,2000,2000,2000
Mid,2000,2000,2000


In [227]:
test_dev_final_df.groupby("EvoAug_Score_dev_bin").count()


,Sequence,Real_Score_dev,EvoAug_Score_dev
EvoAug_Score_dev_bin,,,
High,2000,2000,2000
Low,2000,2000,2000
Mid,2000,2000,2000


In [228]:
y_train = predict(pytorch_model, train_library_x, batch_size=256, func=lambda x: x['mean'])


pearson = [stats.pearsonr(train_library_y[:, i], y_train[:, i])[0] for i in range(train_library_y.shape[1])]
spearman = [stats.spearmanr(train_library_y[:, i], y_train[:, i])[0] for i in range(train_library_y.shape[1])]

print(f'Pearson: {pearson}')
print(f'Spearman: {spearman}')
print(f'Mean Pearson: {np.mean(pearson):.4f}')
print(f'Mean Spearman: {np.mean(spearman):.4f}')



Pearson: [0.7138150435005269, 0.7803728928184888]
Spearman: [0.669070094893995, 0.602529840507455]
Mean Pearson: 0.7471
Mean Spearman: 0.6358


In [199]:
train_library_df = pd.DataFrame({"Sequence": None, "Real_Score_dev": train_library_y[:, 0], "Real_Score_Hk": train_library_y[:, 1], "EvoAug_Score_dev": y_train[:, 0], "EvoAug_Score_Hk": y_train[:, 1]}, index=range(train_library_x.shape[0]))

for i in range(train_library_x.shape[0]):
    try:
        t = characters(train_library_x[i])
        train_library_df.loc[i, "Sequence"] = t
    except:
        print(f"Error at {i}")


print(len(train_library_df))

Error at 16162
Error at 110419
Error at 110420
Error at 110486
Error at 198408
Error at 198692
Error at 200461
Error at 200462
Error at 201102
Error at 217310
Error at 311567
Error at 311568
Error at 311634
Error at 399556
Error at 399840
Error at 401609
Error at 401610
Error at 402250
402296


In [220]:
print(len(train_library_df))
train_library_df = train_library_df[~train_library_df["Sequence"].isna()]
print(len(train_library_df))



402296
402278


In [221]:
train_library_df = add_bin(train_library_df)

valid_bins = ["Low", "Mid", "High"]

binned_train_library_hk = train_library_df[train_library_df['EvoAug_Score_Hk_bin'].isin(valid_bins)]
binned_train_library_dev = train_library_df[train_library_df['EvoAug_Score_dev_bin'].isin(valid_bins)]

Hk_binned_train_library = binned_train_library_hk[["Sequence", "Real_Score_Hk", "EvoAug_Score_Hk", "EvoAug_Score_Hk_bin"]]
Dev_binned_train_library = binned_train_library_dev[["Sequence", "Real_Score_dev", "EvoAug_Score_dev", "EvoAug_Score_dev_bin"]]

print(len(binned_train_library_hk))
print(len(binned_train_library_dev))


121434
122128


In [229]:
hk_low = Hk_binned_train_library[Hk_binned_train_library['EvoAug_Score_Hk_bin'] == "Low"].sample(n=1000)
hk_mid = Hk_binned_train_library[Hk_binned_train_library['EvoAug_Score_Hk_bin'] == "Mid"].sample(n=1000)
hk_high = Hk_binned_train_library[Hk_binned_train_library['EvoAug_Score_Hk_bin'] == "High"].sample(n=1000)

dev_low = Dev_binned_train_library[Dev_binned_train_library['EvoAug_Score_dev_bin'] == "Low"].sample(n=1000)
dev_mid = Dev_binned_train_library[Dev_binned_train_library['EvoAug_Score_dev_bin'] == "Mid"].sample(n=1000)
dev_high = Dev_binned_train_library[Dev_binned_train_library['EvoAug_Score_dev_bin'] == "High"].sample(n=1000)

print(len(hk_low)+len(hk_mid)+len(hk_high))
print(len(dev_low)+len(dev_mid)+len(dev_high))

train_hk_final_df = pd.concat([hk_low, hk_mid, hk_high])
train_dev_final_df = pd.concat([dev_low, dev_mid, dev_high])

train_hk_final_df.groupby("EvoAug_Score_Hk_bin").count()



3000
3000


,Sequence,Real_Score_Hk,EvoAug_Score_Hk
EvoAug_Score_Hk_bin,,,
High,1000,1000,1000
Low,1000,1000,1000
Mid,1000,1000,1000


In [230]:
train_dev_final_df.groupby("EvoAug_Score_dev_bin").count()


,Sequence,Real_Score_dev,EvoAug_Score_dev
EvoAug_Score_dev_bin,,,
High,1000,1000,1000
Low,1000,1000,1000
Mid,1000,1000,1000


In [231]:
Hk_final_df = pd.concat([train_hk_final_df, test_hk_final_df])
Dev_final_df = pd.concat([train_dev_final_df, test_dev_final_df])

Hk_final_df.groupby("EvoAug_Score_Hk_bin").count()


,Sequence,Real_Score_Hk,EvoAug_Score_Hk
EvoAug_Score_Hk_bin,,,
High,3000,3000,3000
Low,3000,3000,3000
Mid,3000,3000,3000


In [232]:
Dev_final_df.groupby("EvoAug_Score_dev_bin").count()


,Sequence,Real_Score_dev,EvoAug_Score_dev
EvoAug_Score_dev_bin,,,
High,3000,3000,3000
Low,3000,3000,3000
Mid,3000,3000,3000


In [ ]:
Hk_final_df["Task"] = "Hk"
Dev_final_df["Task"] = "Dev"


,Sequence,Real_Score_dev,EvoAug_Score_dev,EvoAug_Score_dev_bin,Task
82186,CGGCGGCGAGGCACTGCATCCTTTTCTCTGGAGCGACGTGAGCTGC...,-0.246562,-1.045437,Low,Dev
86477,GTCGCGGACAGACCGAATATCATCCTTTCACCGTCCTGGTGCCTAA...,-1.144817,-1.546986,Low,Dev
134062,GAGACGATGCTCCCCGCGCCGTCTTCCCATCGATTGTGGGACGTCC...,-1.459833,-1.366435,Low,Dev
116664,CGCTGGGCAACGGCGGATCAGCCCACACCTGCCCGCCAATATGTGG...,-2.572726,-1.159021,Low,Dev
312984,CCAGTTGGCCAGCACCCAACGACCATCCCATGACTGCGCACAATCC...,-0.290956,-1.311595,Low,Dev
...,...,...,...,...,...
2539,GCCCAGGAAGGCCATTTTAATGCCTTGTCTGCCTTCAGATCGGATA...,2.881899,2.420901,High,Dev
11037,TTTCACCACGCACGAATACGATTACGAATAACGAATACGAACGTAG...,4.301300,4.012281,High,Dev
20604,TCGTGCTTGCGCTTAATGAACTCCTGCTCCTCGTTGGACAAAACCA...,3.570503,2.061011,High,Dev
4732,CGGATGCCAGCGCAGCAGCTGCCACTGACAAGACAATTTTCACGCA...,7.039060,2.327623,High,Dev


In [ ]:
Hk_final_df.to_csv("Binned_libraries/Hk_final_df.csv")
Dev_final_df.to_csv("Binned_libraries/Dev_final_df.csv")
Combined_final_df = pd.concat([Hk_final_df, Dev_final_df])
Combined_final_df.to_csv("Binned_libraries/Combined_final_df.csv")
print(len(Combined_final_df))


In [252]:
print(len(Combined_final_df))


18000


In [254]:
Hk_final_df.head()

,Sequence,Real_Score_Hk,EvoAug_Score_Hk,EvoAug_Score_Hk_bin,Task
287455,CACATCCACCAGCTTACAAAAGTATGCGCCGACATATATATATAAA...,-0.714100,-1.120561,Low,Hk
41729,AGCAGGCATGTGGCAAGTAGCAAAACTAGCAGCACATCCATTCCAA...,-0.242478,-1.112688,Low,Hk
348462,TAGTTTAAGCCCTATTTATGTACGTCCGATGCGGTTGAAAATAAAA...,-0.813635,-1.040152,Low,Hk
308155,GCCGGAGACCGATGTGTGGACAAGGGCGGGTGCGGCGACGGCGGCC...,-0.550601,-1.076290,Low,Hk
225128,CAAGGGATATTTATTGGAACTTCTTATAGCGGGTTTTATGCTTTTA...,-1.405261,-1.026222,Low,Hk
